# ArcGIS Online Web Application Audit

Description: This will audit all of the web applications in an organization.
 
Created on: 2/13/2018
 
Purpose: This script will audit all of the web applications in an organization, it will also identify the web map used in the web map, then it will compare the sharing between the web map and app, and then write the results to Excel. It will also produce another excel sheet that will identify all of the web maps that are not used in applications.  
 
Authored by: Brandon Longenberger 

Edited By: Joe Guzi
 
Previous Production Date: 8/12/24    

Production Date: 3/5/24

Notes:
- 8/12/24 --- Added Audit Template.
- 3/5/25 --- Added optional code to upload results to AGO

# Step 1: Import Libraries

In [ ]:
# import
print("Import Libraries...")
from arcgis.gis import GIS
from arcgis.gis import ItemProperties
from arcgis.gis import ItemTypeEnum
import getpass
from IPython.display import display
import json
import requests
import csv
from tkinter.filedialog import asksaveasfilename
print("Import Libraries Complete!")

# Step 2: Connect to ArcGIS Online or ArcGIS Enterprise

Use one of the methods below to conenct to ArcGIS Online or ArcGIS Enterprise.

## Connect with current AGO Credentials to this organization

Use this option if you are already signed into AGO and are running this notebook as a hosted notebook in AGO.

In [ ]:
gis = GIS("home")
gis
print("Connect to ArcGIS Online Complete!")

## Connect to ArcGIS Online by providing Org URL and Credentials (built-in users)

In [ ]:
# Connection Variables
Organization = input("What's your oranization? ")
User = input("What's your  username? ")
Password = getpass.getpass("What's your password? ")

# Connection 
gis = GIS(Organization, User, Password)
gis
print("Connected to AGO")

## Connect to ArcGIS Online by providing a profile

We connect by using a profile. This profile has been setup beforehand by using built-in credentials and registering it with the profile itself. We have a section in the documentation that shows you how to do this if you are interested.

In [ ]:
# Connection Variables
gis = GIS(profile="your_online_profile") # edit "your_online_profile" with the name of your profile
gis
print("Connected to AGO")

## User authentication with OAuth 2.0

https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/#user-authentication-with-oauth-20

In [ ]:
# Connection Variables
Organization = input("What's your organization? ")
OrgClientID = input("What's your  ClientID? ")
gis = GIS(Organization, client_id=OrgClientID)
print("Successfully logged in as: " + gis.properties.user.username)
print("Connected to AGO")

## Connect to ArcGIS Enterprise with Single Sign On

In [ ]:
# Connection Variables
Organization = input("What's your oranization? ")
gis = GIS(Organization)
gis
print("Logged in as " + gis.properties.user.username + " to " + gis.properties.portalName)
print("Connected to AGE")

# Step 3: Define Functions

In [ ]:
print("Begin Function Section.")
# This is where you define Functions
def WriteMapInfo(MapID):
    global s
    MapItemList = gis.content.search(query='id:' + str(MapID))
    for MapItem in MapItemList:
        s += str(Item.title.replace(",", "")) + ","
        s += str(Item.id) + ","
        s += str(Item.type) + ","
        s += str(Item.owner) + ","
        s += str(Item.access) + ","
        s += str(MapItem.title.replace(",", "")) + ","
        s += str(MapID) + ","
        s += str(MapItem.type) + ","
        s += str(MapItem.owner) + ","
        s += str(MapItem.access) + ","
        s += ","
        if Item.access == MapItem.access:
            s += "Yes" + ","
        else:
            s += "No" + ","
        s += "\n"

        AddToMapList(MapID)

def GetMedia(Story, Item):
    global s
    Media = Story.get("media")
    if Media != None:
        Type = Media.get("type")
        if Type == "webpage":
            s += str(Item.title.replace(",", "")) + ","
            s += str(Item.id) + ","
            s += str(Item.type) + ","
            s += str(Item.owner) + ","
            s += ","
            s += ","
            s += "Web Page" +","
            s += ","
            s += ","
            s += ","
            Webpage = Media[Type]
            Url = Webpage["url"]
            s += str(Url) + ","
            s += "\n"
        if Type == "webmap":
            Webmap = Media[Type]
            WriteMapInfo(Webmap["id"])

def LoopWebMaps(List):
    for MapID in List:
        WriteMapInfo(MapID)

def AddToMapList(MapID):
    global MapList

    if MapID not in MapList:
        MapList += [MapID]
print("Function Section Complete!")

# Step 4: Execute Code Section

In [ ]:
print("Begin Execute Code Section.") 
WebAppAudit = gis.content.search(query="", item_type="Web Mapping Application", max_items = 10000)
WebMapAudit = gis.content.search(query="", item_type="Web Map", max_items = 10000)
s = ""
t = ""
MapList = []

for Item in WebAppAudit:
    #print(Item)
    print(Item.id)
    print(Item.title)
    # The ArcGIS Knowledge Studio app on AGE is causing the Script to break... I'm bypassing it
    if Item.title != "ArcGIS Knowledge Studio":
        JSON = Item.get_data(try_json=False)
        #print(JSON)
        if JSON != None:
            ItemData = json.loads(JSON)
            Map = ItemData.get("map")
            if Map != None:
                MapItemID = Map.get("itemId")
                WriteMapInfo(MapItemID)
            Values = ItemData.get("values")
            if Values != None:
                WebMap = Values.get("webmap")
                if WebMap != None:
                    if isinstance(WebMap, str):
                        Find = WebMap.find(",")
                        if Find == -1:
                            WriteMapInfo(WebMap)
                        else:
                            Split = WebMap.split(",")
                            LoopWebMaps(Split)
                    elif isinstance(WebMap, list):
                        LoopWebMaps(WebMap)
                Story = Values.get("story")
                if Story != None:
                    Sections = Story.get("sections")
                    if Sections != None:
                        for Section in Sections:
                            GetMedia(Section, Item)
                    Entries = Story.get("entries")
                    if Entries != None:
                        for Entry in Entries:
                            GetMedia(Entry, Item)
        
        else:
            s += str(Item.title.replace(",", "")) + ","
            s += str(Item.id) + ","
            s += str(Item.type) + ","
            s += str(Item.owner) + ","
            s += "\n"
    else:
        print("Ignore: ArcGIS Knowledge Studio")
        
for Map in WebMapAudit:
    Title = str(Map.title.replace(",", ""))
    ID = str(Map.id)
    Owner = str(Map.owner)
    Access = str(Map.access)
    if Map.id in MapList:
        App = "Yes"
    else:
        App = "No"
    t += Title + ","
    t += ID + ","
    t += Owner + ","
    t += Access + ","
    t += App + ","
    t += "\n"
    
print("Execute Code Section Complete!") 

# Step 5: Write the Results to Excel (Optional)

## App and Map details

In [ ]:
print("Begin Write Results to Excel Section - Apps and Map Details") 
AppAndMapfilename = asksaveasfilename() # show an "Open" dialog box and return the path to the selected file
print(AppAndMapfilename)
AppAndMapcsvfile = AppAndMapfilename + ".csv"
print(AppAndMapcsvfile)
with open(AppAndMapcsvfile, 'w') as output:
    print ("Wrtie data To CSV File")
    output.write("AppTitle,AppID,ItemType,Owner,AppAccess,MapTitle,MapID,ItemType,Owner,MapAccess,WebPageUrl,EqualSharing\n")
    output.writelines(s)
    print("Wrtie data To CSV File Complete!")

## Maps with apps

In [ ]:
MapsWithAppsfilepath = asksaveasfilename() # show an "Open" dialog box and return the path to the selected file
print(MapsWithAppsfilename)
MapsWithAppscsvfile = MapsWithAppsfilepath + ".csv"
print(MapsWithAppscsvfile)
with open(MapsWithAppscsvfile, 'w') as output:
    print ("Wrtie data To CSV File")
    output.write("MapTitle,MapID,Owner,Access,Has Application\n")
    output.writelines(t)
    print("Wrtie data To CSV File Complete!")

# Step 6: Upload to AGO (Optional)

## Upload App and Map Details CSV

In [ ]:
print("Begin Upload to AGO Section.") 
AppAndMapfile = AppAndMapfilename.rpartition('/')
#print(file)
AppAndMapfilename = AppAndMapfile[2]
#print(file[2])
testfile = gis.content.search(query="title:"+AppAndMapfilename, item_type="Feature Layer Collection")
if testfile:
    print("Found the service")
    testfile = testfile[0]
    display(testfile)
    from arcgis.features import FeatureLayerCollection
    testfile_collection = FeatureLayerCollection.fromitem(testfile)
    print("The table already exists")
    testfile_collection.manager.overwrite(csvfile)
    display
else:
    print("We have to upload it")
    print("Define User")
    User = gis.users.me.username
    Folder = input("What folder would you like to create your item?")
    # Check if Folder Exists
    FolderExists = gis.content.folders.get(folder=Folder, owner=User)
    print(FolderExists)
    if FolderExists == None:
        print("Create Folder")
        gis.content.folders.create(Folder)
    ItemFolder = gis.content.folders.get(folder=Folder, owner=User)
    print("Define Item Properties")
    item_props = ItemProperties(title=AppAndMapfilename + "_csv", item_type=ItemTypeEnum.CSV.value)
    print("Add csv to AGO")
    csv_future_item = ItemFolder.add(item_properties=item_props, file=MapsWithAppscsvfile)
    csv_item = csv_future_item.result()
    display(csv_item)
    print("Publish csv as Table Layer")
    csv_lyr = csv_item.publish()
    display(csv_lyr)
    
print ("The CSV has been published.\nAll done, thanks for playing!")

## Upload Maps with apps CSV

In [ ]:
print("Begin Upload to AGO Section.") 
MapsWithAppsfile = MapsWithAppsfilepath.rpartition('/')
#print(file)
MapsWithAppsfilename = MapsWithAppsfile[2]
#print(file[2])
testfile = gis.content.search(query="title:"+MapsWithAppsfilename, item_type="Feature Layer Collection")
if testfile:
    print("Found the service")
    testfile = testfile[0]
    display(testfile)
    from arcgis.features import FeatureLayerCollection
    testfile_collection = FeatureLayerCollection.fromitem(testfile)
    print("The table already exists")
    testfile_collection.manager.overwrite(csvfile)
    display
else:
    print("We have to upload it")
    print("Define User")
    User = gis.users.me.username
    Folder = input("What folder would you like to create your item?")
    # Check if Folder Exists
    FolderExists = gis.content.folders.get(folder=Folder, owner=User)
    print(FolderExists)
    if FolderExists == None:
        print("Create Folder")
        gis.content.folders.create(Folder)
    ItemFolder = gis.content.folders.get(folder=Folder, owner=User)
    print("Define Item Properties")
    item_props = ItemProperties(title=MapsWithAppsfilename + "_csv", item_type=ItemTypeEnum.CSV.value)
    print("Add csv to AGO")
    csv_future_item = ItemFolder.add(item_properties=item_props, file=MapsWithAppscsvfile)
    csv_item = csv_future_item.result()
    display(csv_item)
    print("Publish csv as Table Layer")
    csv_lyr = csv_item.publish()
    display(csv_lyr)
    
print ("The CSV has been published.\nAll done, thanks for playing!")